In [58]:
import math
class Co_Occurence_Capturer:

    def __init__(self):
        self.co_occurences = {}
        
    def _assign_entrys(self,word_ids,context_ids):
        for word_id in word_ids:
                for context_id in context_ids:
                    tuple = (word_id,context_id)
                    print(tuple)
                    if tuple in self.co_occurences:
                        self.co_occurences[tuple] += 1
                    else:
                        self.co_occurences[tuple] = 1


    def build_co_occurence2(self,corpus, vocab_local, window_size,amount_split):
        window_size = window_size * 2 + 1 #to bo compliant to GloVe
        window_half = int(window_size/2.0)#also index of center word
    
        #co_occurence = np.zeros((len(vocab_local.word2Id),len(vocab_local.word2Id)), dtype=int)
        block_length = math.ceil(vocab_local.get_size()/amount_split)
        vocab_local.setBlock(amount_split,block_length)
    
        for x in range(0,amount_split):
            for y in range(0,amount_split):
                rest_block_x = vocab_local.get_size() - block_length * x
                rest_block_y = vocab_local.get_size() - block_length * y
            
                #local_matrix = self.load_co_occurence(x,y,block_length)
                #local_matrix = np.zeros((min(block_length,rest_block_x),min(block_length,rest_block_y)), dtype=int)
            
                #go over corpus
                rolling_ids = []
                if (len(corpus) <= (2 * window_half)):#window to short
                    print('shorty morty')
                    for context_word in range(0,len(corpus)):
                        new_ids = vocab_local.get_contrained_ids_text(corpus[context_word],y)
                        rolling_ids.append(new_ids)
                    
                    for word_index in range(0,len(corpus)):
                        current_size_right = word_index + window_half + 1
                        cur_window = corpus[0:current_size_right]#left is zero, due to not being big enougth
                        print('word_index ' + str(word_index))
                        print(corpus[word_index])
                        word_ids = vocab_local.get_contrained_ids_text(corpus[word_index],x)
                        
                        for context_index in range(0,len(cur_window)): 
                            context_ids = rolling_ids[context_index]
                            self._assign_entrys(word_ids,context_ids)
                #else case
                for i in range(0,len(corpus)- (window_half)):
                    window = corpus[i:i+window_size]
                
                    if(i == 0):#fill rolling ids
                        for start in range(0,window_size):
                            new_ids = vocab_local.get_contrained_ids_text(window[start],y)
                            rolling_ids.append(new_ids)
                    else:
                        rolling_ids = rolling_ids[1:]
                        new_ids = vocab_local.get_contrained_ids_text(window[-1],y)
                        rolling_ids.append(new_ids)
                    
                    #do the capture                            
                    center_word = window[window_half]
                    if(i == 0):
                        #loops first positions and then ids
                        for word_index in range(0,window_half):
                            current_size_right = word_index + window_half + 1
                            cur_window = window[0:current_size_right]
                            word_ids = vocab_local.get_contrained_ids_text(window[word_index],x)
                            
                            print('word_index ' + str(word_index))
                            print(window[word_index])
                        
                            for context_index in range(0,len(cur_window)): 
                                context_ids = rolling_ids[context_index]
                                self._assign_entrys(word_ids,context_ids)
                
                    feature_ids = vocab_local.get_contrained_ids_text(center_word,x)
                    print('middle part')
                    print(center_word)
                    for context_index in range(0,len(window)):
                        context_ids = rolling_ids[context_index]
                        self._assign_entrys(feature_ids,context_ids)
                            
                    if(i == len(corpus)- (window_half) - 1):#the last normal
                        print(window_half)
                        print(len(corpus))
                        print(i)
                        print((2 * window_half) - 1)
                        for end in range(0,window_half):
                            cur_window = window[end+1:]#cut index + 1 off
                            end_ids = vocab_local.get_contrained_ids_text(window[-(window_half-end)],x)
                                
                            print('stupid index stuff')
                            print(window[-(window_half-end)])
                            
                            for context_index in range(0,len(cur_window)): 
                                context_ids = rolling_ids[end+1+context_index]
                                self._assign_entrys(end_ids,context_ids)
                                
                        
                            
        return self.co_occurences
    def save_coocurrences(self,file_name):
        with open(file_name, 'wb') as file:
            cloudpickle.dump(self.co_occurences, file)
        self.co_occurences = {}

    def load_co_occurence(self,name):
        with open(name, 'rb+') as file:
            self.co_occurences = cloudpickle.load(file)

In [59]:
import sys
sys.path.append('..')
from Vocabulary import *

c = Co_Occurence_Capturer()
vocab = Vocabulary()
vocab.load('../vocabs/tmp')

c.build_co_occurence2('This is a short test sentance.'.split(),vocab,8,1)

shorty morty
word_index 0
This
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
word_index 1
is
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
word_index 2
a
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
word_index 3
short
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
word_index 4
test
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
word_index 5
sentance.
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)


{(0, 0): 1,
 (0, 1): 1,
 (0, 2): 1,
 (0, 3): 1,
 (0, 4): 1,
 (0, 5): 1,
 (1, 0): 1,
 (1, 1): 1,
 (1, 2): 1,
 (1, 3): 1,
 (1, 4): 1,
 (1, 5): 1,
 (2, 0): 1,
 (2, 1): 1,
 (2, 2): 1,
 (2, 3): 1,
 (2, 4): 1,
 (2, 5): 1,
 (3, 0): 1,
 (3, 1): 1,
 (3, 2): 1,
 (3, 3): 1,
 (3, 4): 1,
 (3, 5): 1,
 (4, 0): 1,
 (4, 1): 1,
 (4, 2): 1,
 (4, 3): 1,
 (4, 4): 1,
 (4, 5): 1,
 (5, 0): 1,
 (5, 1): 1,
 (5, 2): 1,
 (5, 3): 1,
 (5, 4): 1,
 (5, 5): 1}

In [61]:
c = Co_Occurence_Capturer()
c.build_co_occurence2('This is a short test sentance. test test'.split(),vocab,3,1)

word_index 0
This
(0, 0)
(0, 1)
(0, 2)
(0, 3)
word_index 1
is
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
word_index 2
a
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
middle part
short
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 4)
middle part
test
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(4, 4)
(4, 4)
middle part
sentance.
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 4)
(5, 4)
middle part
test
(4, 3)
(4, 4)
(4, 5)
(4, 4)
(4, 4)
middle part
test
(4, 4)
(4, 5)
(4, 4)
(4, 4)
3
8
4
5
stupid index stuff
sentance.
(5, 5)
(5, 4)
(5, 4)
stupid index stuff
test
(4, 4)
(4, 4)
stupid index stuff
test
(4, 4)


{(0, 0): 1,
 (0, 1): 1,
 (0, 2): 1,
 (0, 3): 1,
 (1, 0): 1,
 (1, 1): 1,
 (1, 2): 1,
 (1, 3): 1,
 (1, 4): 1,
 (2, 0): 1,
 (2, 1): 1,
 (2, 2): 1,
 (2, 3): 1,
 (2, 4): 1,
 (2, 5): 1,
 (3, 0): 1,
 (3, 1): 1,
 (3, 2): 1,
 (3, 3): 1,
 (3, 4): 2,
 (3, 5): 1,
 (4, 1): 1,
 (4, 2): 1,
 (4, 3): 2,
 (4, 4): 12,
 (4, 5): 3,
 (5, 2): 1,
 (5, 3): 1,
 (5, 4): 5,
 (5, 5): 2}